In [ ]:
#load libraries
library(Seurat)
#library(tidyverse)
library(ggplot2)
library(scales)
library(dplyr)
library(openxlsx)
library(HGNChelper)
set.seed(411)
print("m")

library(patchwork)

 
#seur_obj <- readRDS("quobyte/from-lssc0/projects/NCR_scRNAseq/results/seurat/harmony_22jan_.rds") #reg processing + wnn
seur_obj <- readRDS("/quobyte/bmhenngrp/from-lssc0/projects/NCR_scRNAseq/results/seurat/3_harmonize_batches_wgs/raw_merge_all_batches_harm_annotated_all_res5_pca50.rds") #reg processing + wnn

    #NCR_scRNAseq/results/seurat/

print("read file")
head(seur_obj)


# load gene set preparation function
source("https://raw.githubusercontent.com/IanevskiAleksandr/sc-type/master/R/gene_sets_prepare.R")
# load cell type annotation function
source("https://raw.githubusercontent.com/IanevskiAleksandr/sc-type/master/R/sctype_score_.R")


# DB file
db_ <- "https://raw.githubusercontent.com/IanevskiAleksandr/sc-type/master/ScTypeDB_full.xlsx";
tissue <- "Immune system" # e.g. Immune system,Pancreas,Liver,Eye,Kidney,Brain,Lung,Adrenal,Heart,Intestine,Muscle,Placenta,Spleen,Stomach,Thymus 

# prepare gene sets
gs_list <- gene_sets_prepare(db_, tissue)



# check Seurat object version (scRNA-seq matrix extracted differently in Seurat v4/v5)
seurat_package_v5 <- isFALSE('counts' %in% names(attributes(seur_obj[["RNA"]])));
print(sprintf("Seurat object %s is used", ifelse(seurat_package_v5, "v5", "v4")))

# extract scaled scRNA-seq matrix
scRNAseqData_scaled <- if (seurat_package_v5) as.matrix(seur_obj[["RNA"]]$scale.data) else as.matrix(seur_obj[["RNA"]]@scale.data)

# run ScType
es.max <- sctype_score(scRNAseqData = scRNAseqData_scaled, scaled = TRUE, gs = gs_list$gs_positive, gs2 = gs_list$gs_negative)

# NOTE: scRNAseqData parameter should correspond to your input scRNA-seq matrix. For raw (unscaled) count matrix set scaled = FALSE
# When using Seurat, we use "RNA" slot with 'scale.data' by default. Please change "RNA" to "SCT" for sctransform-normalized data,
# or to "integrated" for joint dataset analysis. To apply sctype with unscaled data, use e.g. seur_obj[["RNA"]]$counts or seur_obj[["RNA"]]@counts, with scaled set to FALSE.

# merge by cluster
cL_resutls <- do.call("rbind", lapply(unique(seur_obj@meta.data$seurat_clusters), function(cl){
    es.max.cl = sort(rowSums(es.max[ ,rownames(seur_obj@meta.data[seur_obj@meta.data$seurat_clusters==cl, ])]), decreasing = !0)
    head(data.frame(cluster = cl, type = names(es.max.cl), scores = es.max.cl, ncells = sum(seur_obj@meta.data$seurat_clusters==cl)), 10)
}))
sctype_scores <- cL_resutls %>% group_by(cluster) %>% top_n(n = 1, wt = scores)  

# set low-confident (low ScType score) clusters to "unknown"
sctype_scores$type[as.numeric(as.character(sctype_scores$scores)) < sctype_scores$ncells/4] <- "Unknown"
print(sctype_scores[,1:3])

In [ ]:
seur_obj@meta.data$sctype_classification_man = ""
for(j in unique(sctype_scores$cluster)){
  cl_type = sctype_scores[sctype_scores$cluster==j,]; 
  seur_obj@meta.data$sctype_classification_man[seur_obj@meta.data$seurat_clusters == j] = as.character(cl_type$type[1])
}

DimPlot(seur_obj, reduction = "umap", label = TRUE, repel = TRUE, group.by = 'sctype_classification_man')        


In [ ]:
# load libraries
lapply(c("ggraph","igraph","tidyverse", "data.tree"), library, character.only = T)

In [ ]:
# prepare edges
cL_resutls <- cL_resutls[order(cL_resutls$cluster),]; edges = cL_resutls; edges$type = paste0(edges$type,"_",edges$cluster); edges$cluster = paste0("cluster ", edges$cluster); edges = edges[,c("cluster", "type")]; colnames(edges) = c("from", "to"); rownames(edges) <- NULL

# prepare nodes
nodes_lvl1 <- sctype_scores[,c("cluster", "ncells")]; nodes_lvl1$cluster = paste0("cluster ", nodes_lvl1$cluster); nodes_lvl1$Colour = "#f1f1ef"; nodes_lvl1$ord = 1; nodes_lvl1$realname = nodes_lvl1$cluster; nodes_lvl1 = as.data.frame(nodes_lvl1); nodes_lvl2 = c(); 
ccolss <- c("#5f75ae","#92bbb8","#64a841","#e5486e","#de8e06","#eccf5a","#b5aa0f","#e4b680","#7ba39d","#b15928","#ffff99", "#6a3d9a","#cab2d6","#ff7f00","#fdbf6f","#e31a1c","#fb9a99","#33a02c","#b2df8a","#1f78b4","#a6cee3")
for (i in 1:length(unique(cL_resutls$cluster))){
  dt_tmp = cL_resutls[cL_resutls$cluster == unique(cL_resutls$cluster)[i], ]; nodes_lvl2 = rbind(nodes_lvl2, data.frame(cluster = paste0(dt_tmp$type,"_",dt_tmp$cluster), ncells = dt_tmp$scores, Colour = ccolss[i], ord = 2, realname = dt_tmp$type))
}
nodes <- rbind(nodes_lvl1, nodes_lvl2); nodes$ncells[nodes$ncells<1] = 1;
files_db <- openxlsx::read.xlsx(db_)[,c("cellName","shortName")]; files_db = unique(files_db); nodes = merge(nodes, files_db, all.x = T, all.y = F, by.x = "realname", by.y = "cellName", sort = F)
nodes$shortName[is.na(nodes$shortName)] = nodes$realname[is.na(nodes$shortName)]; nodes = nodes[,c("cluster", "ncells", "Colour", "ord", "shortName", "realname")]

mygraph <- graph_from_data_frame(edges, vertices=nodes)

# Make the graph
gggr <- ggraph(mygraph, layout = 'circlepack', weight=I(ncells)) + 
  geom_node_circle(aes(filter=ord==1,fill=I("#F5F5F5"), colour=I("#D3D3D3")), alpha=0.9) + geom_node_circle(aes(filter=ord==2,fill=I(Colour), colour=I("#D3D3D3")), alpha=0.9) +
  theme_void() + geom_node_text(aes(filter=ord==2, label=shortName, colour=I("#ffffff"), fill="white", repel = !1, parse = T, size = I(log(ncells,25)*1.5)))+ geom_node_label(aes(filter=ord==1,  label=shortName, colour=I("#000000"), size = I(3), fill="white", parse = T), repel = !0, segment.linetype="dotted")
  
DimPlot(pbmc, reduction = "umap", label = TRUE, repel = TRUE, cols = ccolss)+ gggr

In [ ]:
str(seur_obj)

In [ ]:
# Filter Seurat object
#seur_obj <- subset(seur_obj, subset = !is.na(donor_id))
source("https://raw.githubusercontent.com/IanevskiAleksandr/sc-type/master/R/sctype_wrapper.R"); 
seur_obj <- run_sctype(seur_obj, assay = "RNA", scaled = TRUE, known_tissue_type="Immune system",custom_marker_file="https://raw.githubusercontent.com/IanevskiAleksandr/sc-type/master/ScTypeDB_short.xlsx",name="sctype_classification")

#unique(seur_obj$sctype_classification)

In [ ]:
head(seur_obj@meta.data)

In [10]:
seur_obj@meta.data$sctype_classification <- gsub(
  pattern = "γδ-T cells", 
  replacement = "Gamma-delta T cells", 
  seur_obj@meta.data$sctype_classification
)

In [ ]:
nk_cells <- subset(seur_obj, subset = sctype_classification == 'Natural killer  cells')
head(nk_cells)

In [ ]:
# Visualize the UMAPS
options(repr.plot.width = 20, repr.plot.height = 12)
DimPlot(seur_obj, reduction = 'umap', group.by = "sctype_classification",  label = TRUE, repel = TRUE, label.size = 5.5) 
DimPlot(seur_obj, reduction = 'umap', group.by = "batch",  label = TRUE, repel = TRUE, label.size = 5.5) 

In [ ]:
DotPlot(nk_cells, features = "THBD1", group.by = "sctype_classification") + RotatedAxis()
VlnPlot(seur_obj, features = "THBD1", group.by = "sctype_classification", pt.size = 0)  # Set pt.size > 0 to show individual points

#DimPlot(seur_obj, reduction = 'adt.umap',   label = TRUE, repel = TRUE, label.size = 5.5) 

In [ ]:
# Process RNA data with standard workflow
nk_cells_repro <- NormalizeData(nk_cells, assay = "RNA", normalization.method = "LogNormalize") %>%
  FindVariableFeatures(assay = "RNA", selection.method = "vst", nfeatures = 2000) %>%    # Specify RNA assay for variable feature selection
  ScaleData(assay = "RNA") %>%               # Specify RNA assay for scaling
  RunPCA(assay = "RNA",reduction.name = "pca", reduction.key = "PCA_") %>% # Specify RNA assay for PCA
  FindNeighbors(assay = "RNA", dims = 1:50)  %>%
  FindClusters(assay = "RNA",  resolution = 1.2)  %>%
  RunUMAP(reduction = 'pca', dims = 1:50, assay = 'RNA', 
              reduction.name = 'rna.umap', reduction.key = 'rnaUMAP_')
print("processed combined RNA data")

In [ ]:
print(DimPlot(nk_cells_repro, reduction = "rna.umap", group.by = "batch"))
#print(DimPlot(nk_cells_repro, reduction = "rna.umap", group.by = "percent.mt"))
#print(DimPlot(nk_cells_repro, reduction = "rna.umap", group.by = "percent.ribo"))

In [ ]:
DotPlot(nk_cells_repro, features = c("Hu.CD16", "Hu.CD56", "Hu.CD94", "Hu.CD161"), 
        assay = "ADT") + RotatedAxis()
DotPlot(nk_cells_repro, features = c("KLRC2", "KLRB1"), 
        assay = "RNA") + RotatedAxis()



In [ ]:
#saveRDS(nk_cells_repro, "nk_cells_filtered.rds")
FeaturePlot(nk_cells_repro, features = c("Hu.CD16", "Hu.CD56", "Hu.CD94", "Hu.CD161"), 
            reduction = "rna.umap" )

In [ ]:
FeaturePlot(nk_cells_repro, features = c("FCGR3A",
"NCAM1",
"KLRC2",
"KLRB1"), 
            reduction = "rna.umap" )

In [ ]:
###### grep("IDO1", rownames(seur_obj), value = TRUE, ignore.case = TRUE)
FeaturePlot(seur_obj, features = "IDO1")

#DimPlot(seur_obj, reduction = 'adt.umap',   label = TRUE, repel = TRUE, label.size = 5.5) 

In [ ]:
DimPlot(nk_cells_repro, reduction = 'rna.umap',   label = TRUE, repel = TRUE, label.size = 5.5) 


In [ ]:
print(DimPlot(seur_obj, reduction = "rna.umap", group.by = "batch"))


In [ ]:
print(table(seur_obj$batch, seur_obj$seurat_clusters))

In [ ]:
# TCR clonotype frequency
tcr_freq <- table(seur_obj$t_cdr3s_aa)
#tcr_freq <- table(seur_obj$t_v_gene)
# Display the top 10 clonotypes by frequency

head(sort(tcr_freq, decreasing = TRUE), 10)

# BCR clonotype frequency
bcr_freq <- table(seur_obj$b_cdr3s_aa)
#bcr_freq <- table(seur_obj$b_v_gene)

# Display the top 10 clonotypes by frequency
head(sort(bcr_freq, decreasing = TRUE), 10)

# Identify expanded TCR clonotypes (appears in 10 or more cells)
expanded_tcr <- tcr_freq[tcr_freq >= 10]

# Get the top 10 expanded TCR clonotypes by frequency
top_10_expanded_tcr <- head(sort(expanded_tcr, decreasing = TRUE), 10)
# Display expanded TCR clonotypes
length(expanded_tcr)
length(top_10_expanded_tcr)
# Identify expanded BCR clonotypes (appears in 10 or more cells)
expanded_bcr <- bcr_freq[bcr_freq >= 10]
top_10_expanded_bcr <- head(sort(expanded_bcr, decreasing = TRUE), 10)
# Display expanded TCR clonotypes
length(expanded_bcr)
length(top_10_expanded_bcr)
                     
top_10_tcr_clonotypes = names(top_10_expanded_tcr)
top_10_bcr_clonotypes = names(top_10_expanded_bcr)  
                     
print(top_10_bcr_clonotypes)
print(top_10_tcr_clonotypes)  

                     # Create a data frame combining clonotype and cell type information
tcr_df <- data.frame(
  clonotype = seur_obj$t_cdr3s_aa,
  cell_type = seur_obj$sctype_classification
)
# Filter the data to include only the top 10 clonotypes
tcr_top_10_df <- tcr_df %>% filter(clonotype %in% top_10_tcr_clonotypes)

# Summarize clonotype frequencies by cell type for top 10 clonotypes
tcr_top_10_freq_by_celltype <- tcr_top_10_df %>%
  group_by(cell_type, clonotype) %>%
  summarise(frequency = n()) %>%
  ungroup()

# View the summarized data
head(tcr_top_10_freq_by_celltype)

print(ggplot(tcr_top_10_freq_by_celltype, aes(y = reorder(clonotype, -frequency), x = frequency, fill = cell_type)) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(title = "Top 10 TCR Clonotype Frequency Across Cell Types", y = "Clonotype", x = "Frequency") +
  theme_minimal() +
  theme(
    axis.text.x = element_text(size = 12, face = "bold", angle = 45, hjust = 1),  # Rotate x-axis labels by 45 degrees
    axis.text.y = element_text(size = 14, face = "bold"),
    plot.title = element_text(size = 16, face = "bold"),
    legend.text = element_text(size = 18),  # Increase legend text size
    legend.title = element_text(size = 18)  # Increase legend title size
  ))

# Create a data frame combining clonotype and tcrgene info
tcr_df2 <- data.frame(
  clonotype = seur_obj$t_cdr3s_aa,
  gene = seur_obj$t_v_gene
)
# Filter the data to include only the top 10 clonotypes
tcr_top_10_df2 <- tcr_df2 %>% filter(clonotype %in% top_10_tcr_clonotypes)
 # Summarize clonotype frequencies by cell type for top 10 clonotypes
tcr_top_10_freq_by_gene <- tcr_top_10_df2 %>%
  group_by(gene, clonotype) %>%
  summarise(frequency = n()) %>%
  ungroup()

# View the summarized data
head(tcr_top_10_freq_by_gene)

print(ggplot(tcr_top_10_freq_by_gene, aes(y = reorder(clonotype, -frequency), x = frequency, fill = gene)) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(title = "Top 10 TCR Clonotype Frequency Across Cell Types", y = "Clonotype", x = "Frequency") +
  theme_minimal() +
  theme(
    axis.text.x = element_text(size = 12, face = "bold", angle = 45, hjust = 1),  # Rotate x-axis labels by 45 degrees
    axis.text.y = element_text(size = 14, face = "bold"),
    plot.title = element_text(size = 16, face = "bold"),
    legend.text = element_text(size = 18),  # Increase legend text size
    legend.title = element_text(size = 18)))  # Increase legend title size

In [ ]:
bcr_df <- data.frame(
  clonotype = seur_obj$b_cdr3s_aa,
  cell_type = seur_obj$sctype_classification
)
# Filter the data to include only the top 10 clonotypes
bcr_top_10_df <- bcr_df %>% filter(clonotype %in% top_10_bcr_clonotypes)

# Summarize clonotype frequencies by cell type for top 10 clonotypes
bcr_top_10_freq_by_celltype <- bcr_top_10_df %>%
  group_by(cell_type, clonotype) %>%
  summarise(frequency = n()) %>%
  ungroup()

# View the summarized data
head(bcr_top_10_freq_by_celltype)

print(ggplot(bcr_top_10_freq_by_celltype, aes(y = reorder(clonotype, -frequency), x = frequency, fill = cell_type)) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(title = "Top 10 BCR Clonotype Frequency Across Cell Types", y = "Clonotype", x = "Frequency") +
  theme_minimal() +
  theme(
    axis.text.x = element_text(size = 12, face = "bold", angle = 45, hjust = 1),  # Rotate x-axis labels by 45 degrees
    axis.text.y = element_text(size = 14, face = "bold"),
    plot.title = element_text(size = 16, face = "bold"),
    legend.text = element_text(size = 18),  # Increase legend text size
    legend.title = element_text(size = 18)  # Increase legend title size
  ))


In [ ]:
# Create a data frame combining clonotype and bcrgene info
bcr_df2 <- data.frame(
  clonotype = seur_obj$b_cdr3s_aa,
  gene = seur_obj$b_v_gene
)

# Filter the data to include only the top 10 clonotypes
bcr_top_10_df2 <- bcr_df2 %>% filter(clonotype %in% top_10_bcr_clonotypes)
bcr_top_10_df2 
# Summarize clonotype frequencies by cell type for top 10 clonotypes
bcr_top_10_freq_by_gene <- bcr_top_10_df2 %>%
  group_by(gene, clonotype) %>%
  summarise(frequency = n()) %>%
  ungroup()

# View the summarized data
head(bcr_top_10_freq_by_gene)

print(ggplot(bcr_top_10_freq_by_gene, aes(y = reorder(clonotype, -frequency), x = frequency, fill = gene)) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(title = "Top 10 BCR Clonotype Frequency Across genes", y = "Clonotype", x = "Frequency") +
  theme_minimal() +
  theme(
    axis.text.x = element_text(size = 12, face = "bold", angle = 45, hjust = 1),  # Rotate x-axis labels by 45 degrees
    axis.text.y = element_text(size = 14, face = "bold"),
    plot.title = element_text(size = 16, face = "bold"),
    legend.text = element_text(size = 18),  # Increase legend text size
    legend.title = element_text(size = 18)))  # Increase legend title size
                     

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 12)

RNAUMAP = DimPlot(seur_obj, reduction = "rna.umap", label = TRUE, repel = TRUE, group.by = 'sctype_classification',label.size = 5) + 
  NoLegend()   

p1 = FeaturePlot(seur_obj, features = c("PF4"), reduction = 'rna.umap' )  #PF4, SDPR, GNG11, PPBP.
p2 = FeaturePlot(seur_obj, features = c("GNG11"), reduction = 'rna.umap' )  #PF4, SDPR, GNG11, PPBP.
p3 = FeaturePlot(seur_obj, features = c("PPBP"), reduction = 'rna.umap' )  #PF4, SDPR, GNG11, PPBP.

(RNAUMAP + p1 + p2 + p3) + plot_layout(ncol = 2)

In [ ]:
markers <-read.csv("/share/hennlab/projects/NCR_scRNAseq/data/Markers.csv", na.strings = c("", "NA"))
#head(markers)
#markers[[marker == "CD19"]]$ADT_name
tcell_rna_markers <- na.omit(markers[markers$Cell.type == "T-cells", "RNA_name"])
bcell_rna_markers <- na.omit(markers[markers$Cell.type == "B-cell", "RNA_name"])
monocytes_rna_markers <- na.omit(markers[markers$Cell.type == "Monocytes", "RNA_name"])
nkcell_rna_markers <- na.omit(markers[markers$Cell.type == "NK-cells", "RNA_name"])
dendritic_cell_rna_markers <- na.omit(markers[markers$Cell.type == "Dendritic-Cell", "RNA_name"])


tcell_rna_markers 
bcell_rna_markers
monocytes_rna_markers
nkcell_rna_markers
dendritic_cell_rna_markers

my_colors <- c(
  "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd",
  "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf",
  "#393b79", "#637939", "#8c6d31", "#843c39", "#7b4173",
  "#5254a3", "#9c9ede", "#d6616b", "#b00b69", "#de9ed6"
)
adt_markers <- na.omit(markers$ADT_name)
 
adt_markers


In [ ]:
 #head(markers)
#markers[[marker == "CD19"]]$ADT_name
adt_markers <- na.omit(markers$ADT_name)
 
adt_markers


In [ ]:
DotPlot(seur_obj, features = tcell_rna_markers, group.by = "sctype_classification", cols = my_colors, assay = "RNA") + RotatedAxis()  

In [ ]:
#DotPlot(seur_obj, features = nkcell_rna_markers, group.by = "sctype_classification", cols = my_colors) + RotatedAxis()
DotPlot(seur_obj, features = nkcell_rna_markers, group.by = "sctype_classification") + RotatedAxis()

In [ ]:
DotPlot(seur_obj, features = bcell_rna_markers, group.by = "sctype_classification", cols = my_colors) + RotatedAxis()

In [ ]:
DotPlot(seur_obj, features = dendritic_cell_rna_markers, group.by = "sctype_classification", cols = my_colors) + RotatedAxis()

In [ ]:
DotPlot(seur_obj, features = monocytes_rna_markers, group.by = "sctype_classification", cols = my_colors, assay = "RNA") + RotatedAxis()

In [ ]:
DotPlot(seur_obj, features = adt_markers, group.by = "sctype_classification", assay ="ADT",cols = c("lightgrey", "blue")) + RotatedAxis()
 

In [ ]:
RColorBrewer::brewer.pal.info

In [ ]:
 
ridge_plots <- RidgePlot(
  object = seur_obj,
  features = adt_markers,
  group.by = "sctype_classification",
  assay = "ADT",
  slot = "data",
  cols = my_colors
)

# Arrange plots into 4 columns
ridge_plots + plot_layout(ncol = 2)

In [ ]:
#FeaturePlot(seur_obj, features = c("Hu.CD94"), reduction = 'adt.umap' ) 
DimPlot(seur_obj, reduction = "adt.umap", label = TRUE, repel = TRUE, group.by = 'sctype_classification', label.size = 5) +theme(legend.title = element_text(size = 16),legend.text = element_text(size = 16))     # Adjust legend label size

In [ ]:
 DotPlot(seur_obj, features = adt_markers, group.by = "sctype_classification", assay ="ADT",cols = my_colors, col.min = -1) + RotatedAxis()

In [ ]:
# Set threshold on scaled data
scaled_data <- seur_obj[["RNA"]]$scale.data


# Set threshold for scaled expression (e.g., > 0.5)
scaled_threshold <- 0.5
scaled_features <- apply(scaled_data, 1, function(x) any(x > scaled_threshold))

# Filter features
filtered_features_scaled <- features[scaled_features]

# Plot DotPlot with filtered features
DotPlot(seur_obj, features = filtered_features_scaled) + RotatedAxis()


In [ ]:
str(seur_obj)

In [ ]:
#DotPlot(seur_obj, features = c("Hu.CD8","Hu.CD16"), group.by = "sctype_classification",assay ="ADT", cols = my_colors) + RotatedAxis()


In [ ]:
RidgePlot(seur_obj, "CD8A" , group.by = "sctype_classification")  

In [ ]:
# load libraries
lapply(c("ggraph","igraph", "data.tree"), library, character.only = T)
#lapply(c("ggraph","igraph","tidyverse", "data.tree"), library, character.only = T)


In [ ]:
    reference <- readRDS("/share/hennlab/projects/NCR_scRNAseq/data/pbmc_multimodal_2023.rds")

In [ ]:
anchors <- FindTransferAnchors(
  reference = reference,
  query = seur_obj,
  normalization.method = "SCT",
  reference.reduction = "spca",
  dims = 1:50
)

In [ ]:
seur_obj_mapped <- MapQuery(
  anchorset = anchors,
  query = seur_obj,
  reference = reference,
  refdata = list(
    celltype.l1 = "celltype.l1",
    celltype.l2 = "celltype.l2",
    predicted_ADT = "ADT"
  ),
  reference.reduction = "spca", 
  reduction.model = "wnn.umap"
)

In [ ]:
p1 = DimPlot(seur_obj_mapped, reduction = "ref.umap", group.by = "predicted.celltype.l1", label = TRUE, label.size = 3, repel = TRUE)  
p2 = DimPlot(seur_obj_mapped, reduction = "ref.umap", group.by = "predicted.celltype.l2", label = TRUE, label.size = 3 ,repel = TRUE)  
p1 + p2